In [59]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score, mean_absolute_error
import warnings
warnings.filterwarnings("ignore")


In [4]:
df = pd.read_csv('insurance.csv')
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [9]:
df.shape

(1338, 7)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [11]:
df.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [12]:
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


In [13]:
df.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges'], dtype='object')

In [14]:
num_col = ['age', 'sex', 'children', 'charges']
num_col

['age', 'sex', 'children', 'charges']

In [15]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [5]:
df["sex"].unique()

array(['female', 'male'], dtype=object)

In [6]:
df["sex"].value_counts()

sex
male      676
female    662
Name: count, dtype: int64

In [14]:
encoder = LabelEncoder()
df['Sex_encoded'] = encoder.fit_transform(df['sex'])
df['Sex_encoded']

0       0
1       1
2       1
3       1
4       1
       ..
1333    1
1334    0
1335    0
1336    0
1337    0
Name: Sex_encoded, Length: 1338, dtype: int64

In [15]:
df.head()

,age,sex,bmi,children,smoker,region,charges,Sex_encoded
0,19,female,27.900,0,yes,southwest,16884.92400,0
1,18,male,33.770,1,no,southeast,1725.55230,1
2,28,male,33.000,3,no,southeast,4449.46200,1
3,33,male,22.705,0,no,northwest,21984.47061,1
4,32,male,28.880,0,no,northwest,3866.85520,1


In [16]:
df = df.drop('sex', axis=1)

In [19]:
df.rename(columns={"Sex_encoded": "Sex"}, inplace=True)
df.head()

,age,bmi,children,smoker,region,charges,Sex
0,19,27.900,0,yes,southwest,16884.92400,0
1,18,33.770,1,no,southeast,1725.55230,1
2,28,33.000,3,no,southeast,4449.46200,1
3,33,22.705,0,no,northwest,21984.47061,1
4,32,28.880,0,no,northwest,3866.85520,1


In [20]:
df['region'].value_counts()

region
southeast    364
southwest    325
northwest    325
northeast    324
Name: count, dtype: int64

In [34]:
ohe = OneHotEncoder(sparse_output=False)
transformed = ohe.fit_transform(df[['region']])
column = ohe.get_feature_names_out(['region'])
encoded_df = pd.DataFrame(transformed, columns=column)



In [35]:
df = pd.concat([df, encoded_df], axis=1)
df

,age,bmi,children,smoker,region,charges,Sex,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,yes,southwest,16884.92400,0,0.0,0.0,0.0,1.0
1,18,33.770,1,no,southeast,1725.55230,1,0.0,0.0,1.0,0.0
2,28,33.000,3,no,southeast,4449.46200,1,0.0,0.0,1.0,0.0
3,33,22.705,0,no,northwest,21984.47061,1,0.0,1.0,0.0,0.0
4,32,28.880,0,no,northwest,3866.85520,1,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,no,northwest,10600.54830,1,0.0,1.0,0.0,0.0
1334,18,31.920,0,no,northeast,2205.98080,0,1.0,0.0,0.0,0.0
1335,18,36.850,0,no,southeast,1629.83350,0,0.0,0.0,1.0,0.0
1336,21,25.800,0,no,southwest,2007.94500,0,0.0,0.0,0.0,1.0


In [37]:
df = df.drop('region', axis=1)


In [38]:
df.head()

,age,bmi,children,smoker,charges,Sex,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,yes,16884.92400,0,0.0,0.0,0.0,1.0
1,18,33.770,1,no,1725.55230,1,0.0,0.0,1.0,0.0
2,28,33.000,3,no,4449.46200,1,0.0,0.0,1.0,0.0
3,33,22.705,0,no,21984.47061,1,0.0,1.0,0.0,0.0
4,32,28.880,0,no,3866.85520,1,0.0,1.0,0.0,0.0


In [39]:
df['smoker'].value_counts()

smoker
no     1064
yes     274
Name: count, dtype: int64

In [41]:
df['smoker'] = encoder.fit_transform(df['smoker'])
df['smoker']

0       1
1       0
2       0
3       0
4       0
       ..
1333    0
1334    0
1335    0
1336    0
1337    1
Name: smoker, Length: 1338, dtype: int64

In [42]:
df.head()

,age,bmi,children,smoker,charges,Sex,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,1,16884.92400,0,0.0,0.0,0.0,1.0
1,18,33.770,1,0,1725.55230,1,0.0,0.0,1.0,0.0
2,28,33.000,3,0,4449.46200,1,0.0,0.0,1.0,0.0
3,33,22.705,0,0,21984.47061,1,0.0,1.0,0.0,0.0
4,32,28.880,0,0,3866.85520,1,0.0,1.0,0.0,0.0


In [45]:
X = df.drop('charges', axis=1)
y = df['charges']

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [52]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1070, 9), (268, 9), (1070,), (268,))

In [67]:
#Standard Scaler -Since this is a regression problem, scaling helps models like:
#Linear Regression
#Ridge
#Lasso
#KNN

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_Scaled = scaler.transform(X_test)
X_test_Scaled

array([[ 0.40114007, -0.89153925,  0.73433626, ..., -0.56079971,
        -0.59966106, -0.5723141 ],
       [-0.23863782, -0.08946143, -0.91119211, ...,  1.78316783,
        -0.59966106, -0.5723141 ],
       [ 1.75178229, -0.60845296, -0.91119211, ...,  1.78316783,
        -0.59966106, -0.5723141 ],
       ...,
       [-0.09646495, -0.41972876, -0.08842793, ..., -0.56079971,
        -0.59966106, -0.5723141 ],
       [ 1.04091797,  2.78941026, -0.91119211, ..., -0.56079971,
         1.66760869, -0.5723141 ],
       [ 0.82765867,  0.60252728, -0.08842793, ..., -0.56079971,
        -0.59966106,  1.74729228]], shape=(268, 9))

In [68]:
linear = LinearRegression()
linear.fit(X_train_scaled, y_train)

y_pred_lin = linear.predict(X_test_Scaled)
y_pred_lin

array([ 8969.55027444,  7068.74744287, 36858.41091155,  9454.67850053,
       26973.17345656, 10864.11316424,   170.28084136, 16903.45028662,
        1092.43093614, 11218.34318352, 28101.68455267,  9377.73460205,
        5263.0595179 , 38416.04221107, 40255.82339284, 37098.25353123,
       15240.39392306, 35912.88264434,  9112.52398703, 31461.92108909,
        3847.68845883, 10130.12001517,  2370.54189389,  7140.21550828,
       11301.76782638, 12961.65366224, 14509.47251876,  6159.8976107 ,
        9963.85857263,  2177.85718217,  9115.93673493, 13073.68932159,
        4561.82376202,  3408.20756033,  4459.81359745, 13032.06505076,
        1979.99357292,  8813.28303302, 33271.29124448, 32585.51583927,
        3908.76090964,  4326.10774721, 14142.81326533, 11423.45494846,
        8774.13955311, 12097.28051001,  5281.57353499,  3150.5596042 ,
       35494.46461214,  9150.1124786 , 15836.84575621,  2343.57470069,
       12364.78414194,  1482.29488266, 13389.06105161, 12573.57395972,
      

In [70]:
r2Score = r2_score(y_test, y_pred_lin)
MAE = mean_absolute_error(y_test, y_pred_lin)
r2Score, MAE

(0.7835929767120722, 4181.194473753653)

In [71]:
ridge = Ridge(alpha=1.0)
ridge.fit(X_train_scaled, y_train)
y_pred_ridge = ridge.predict(X_test_Scaled)

lasso = Lasso(alpha=1.0)
lasso.fit(X_train_scaled, y_train)
y_pred_lasso = lasso.predict(X_test_Scaled)

print("Ridge Regression")
print("R2:", r2_score(y_test, y_pred_ridge))
print("MAE:", mean_absolute_error(y_test, y_pred_ridge))

print("Lasso Regression")
print("R2:", r2_score(y_test, y_pred_lasso))
print("MAE:", mean_absolute_error(y_test, y_pred_lasso))

Ridge Regression
R2: 0.7835437156081317
MAE: 4182.753384895231
Lasso Regression
R2: 0.7835697199613387
MAE: 4181.426352782553


In [72]:
n = X_test.shape[0]
p = X_test.shape[1]
adjust_r2 = 1 - ((1 - r2Score) * (n - 1) / (n -p -1)) / (n - p - 1)
adjust_r2  

0.9991319530794742